In [1]:
%load_ext autoreload
%autoreload 2
from user_study_helpers import *

# Load Dev Set and primitive data

Expand a video by clicking on it; then use `;` to play the video.

In [105]:
bounding_boxes = get_bboxes('dev')
parking_space_gt = get_gt( )
visualize_boxes([bounding_boxes, parking_space_gt])




  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 70.80it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 440.12it/s]

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xc4\xbd\xcb\xae$Kr\xae\xf7*\x07=\x16\x04\xbf_4\xd4…

In [77]:
widget = visualize_boxes([bounding_boxes])
display(widget)

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xc4\xbd\xdb\xae$Kr\x9e\xf9*B_\x0f\x06~>\xcc\xe5\xb…

# Task: detect all empty parking spaces

Your goal is to write a Rekall program to detect all empty parking spaces (visualized in the second timeline above).

You're given a Rekall `IntervalSetMapping` object, `bounding_boxes`, that contains detections from Mask R-CNN. The Intervals contain 3D bounds, and the payloads contain the class and the class score:

In [3]:
bounding_boxes[0].get_intervals()[0]

<Interval t1:0.0 t2:30.0 x1:0.0 x2:0.08424050211906434 y1:0.5207680172390408 y2:0.6528446621365017 payload:{'class': 'car', 'score': 0.9638893008232117}>

The bounding boxes are sampled every thirty seconds (hence why the Interval above has time bounds of 0 to 30), and so are the ground truth annotations.

Your goal is to fill in the following function:

```Python
def detect_empty_parking_spaces(bounding_boxes, reference_video):
    """
    Function to detect empty parking spaces.
    
    bounding_boxes is a Rekall IntervalSetMapping object whose intervals are
    bounding box outputs from Mask R-CNN.
    
    first_video_id is the video ID of the first video. We guarantee that at
    time 0 in this video, all the car detections are parking spots.
    
    This function needs to return a Rekall IntervalSetMapping object whose
    Intervals are empty parking spots in the video.
    
    The output Intervals need to have time extent (0, 30), (30, 60), etc.
    Each Interval should have the spatial extent of a single empty parking
    spot.
    """
    
    # Your code here
```

This function takes in bounding box outputs from Mask R-CNN, and detects empty parking spaces. We also pass in the video ID of the first video - we guarantee that at time zero of this video, all the car detections will be parked cars.

This function will be evaluated against an unseen test set at the end of the user study. We'll be using the Average Precision metric.

We provide some helper functions to evaluate your programs on the dev set below!

This task is inspired by [this Medium blog post](https://medium.com/@ageitgey/snagging-parking-spaces-with-mask-r-cnn-and-python-955f2231c400):
* They use an off-the-shelf object detector to detect cars (like what you have in `bounding_boxes`)
* They take a timestamp where all the parking spots are full, and use car detections to get parking spots (like time 0 of `reference_video`)
* Then empty parking spots are just parking spots where there are no cars

In [132]:
def detect_empty_parking_spaces(bounding_boxes, reference_video):
    """
    Function to detect empty parking spaces.
    
    bounding_boxes is a Rekall IntervalSetMapping object whose intervals are
    bounding box outputs from Mask R-CNN.
    
    first_video_id is the video ID of the first video. We guarantee that at
    time 0 in this video, all the car detections are parking spots.
    
    This function needs to return a Rekall IntervalSetMapping object whose
    Intervals are empty parking spots in the video.
    
    The output Intervals need to have time extent (0, 30), (30, 60), etc.
    Each Interval should have the spatial extent of a single empty parking
    spot.
    """
    
    # Your code here

    first_frame = bounding_boxes[reference_video].filter(
        lambda interval: interval['t1'] == 0 and interval['payload']['class'] == 'car'
        ).get_intervals()

    video_lengths = {
        key: bounding_boxes[key].get_intervals()[-1]['t2']
        for key in bounding_boxes
    }

    first_duplicated = IntervalSetMapping({
        key: IntervalSet([
            Interval(Bounds3D(
                t1 = t,
                t2 = t + 30,
                x1 = interval['x1'],
                x2 = interval['x2'],
                y1 = interval['y1'],
                y2 = interval['y2']
            ))
            for t in range(0, int(video_lengths[key]), 30)
            for interval in first_frame
        ])
        for key in bounding_boxes
    })
    
    this_frame = bounding_boxes.filter(lambda interval: interval['payload']['class'] == 'car')
    
    parking_spots = first_duplicated.minus(
        this_frame,
        predicate = and_pred(
            Bounds3D.T(equal()),
            Bounds3D.X(overlaps()),
            Bounds3D.Y(overlaps()),
            iou_at_least(0.1)
        ),
        window = 0.0,
        progress_bar = True
    )
    
    return  parking_spots

In [129]:
empty_parking_spaces = detect_empty_parking_spaces(bounding_boxes, 1)

# Visualize the predictions the first row will be your predictions, second
# row will be the ground truth
widget = visualize_boxes([empty_parking_spaces, parking_space_gt])
display(widget)
ap = compute_ap(empty_parking_spaces, parking_space_gt)
    
print('Average precision: ', ap)




  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.38s/it]


100%|██████████| 2/2 [00:15<00:00,  6.00s/it]

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xed\x9dOo\x1bG\x12\xc5\xbfJ\xa0\xf3\xc2\x9a\xfe\xd…




  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 29.45it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  2.96it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  2.91it/s]

Average precision:  0.6975929324089721


## Helper functions

`evaluate_on_dev` runs `detect_empty_parking_spaces` on `bounding_boxes`, and visualizes the results along with the ground truth in the dev set. Then it computes the AP score on the dev set.

Of course, you can also run these functions yourself!

`compute_ap` takes the predicted parking spaces and the ground truth set, and computes the AP score. As we saw before, `visualize_boxes` takes a list of `IntervalSetMapping` objects and visualizes them.

In [36]:
def evaluate_on_dev():
    empty_parking_spaces = detect_empty_parking_spaces(bounding_boxes, 0)
    
    # Visualize the predictions the first row will be your predictions, second
    # row will be the ground truth
    widget = visualize_boxes([empty_parking_spaces, parking_space_gt])
    display(widget)
    
    # Compute average precision on the dev set
    ap = compute_ap(empty_parking_spaces, parking_space_gt)
    
    print('Average precision: ', ap)

In [131]:
evaluate_on_dev()




  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.49s/it]


100%|██████████| 2/2 [00:15<00:00,  6.08s/it]

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xed\x9dOo\x1bG\x12\xc5\xbfJ\xa0\xf3\xc2\x9a\xfe\xd…




  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 29.57it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  3.01it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  2.93it/s]

Average precision:  0.6975929324089721


In [133]:
evaluate_on_test(detect_empty_parking_spaces)




  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 787.51it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 64.02it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.15s/it]


100%|██████████| 2/2 [00:15<00:00,  5.96s/it]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 71.66it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  5.86it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  7.66it/s]

Average precision:  0.665158371040724


# Post-hoc Analysis

In [138]:
def detect_empty_parking_spaces_modified(bounding_boxes, reference_video):
    """
    Function to detect empty parking spaces.
    
    bounding_boxes is a Rekall IntervalSetMapping object whose intervals are
    bounding box outputs from Mask R-CNN.
    
    first_video_id is the video ID of the first video. We guarantee that at
    time 0 in this video, all the car detections are parking spots.
    
    This function needs to return a Rekall IntervalSetMapping object whose
    Intervals are empty parking spots in the video.
    
    The output Intervals need to have time extent (0, 30), (30, 60), etc.
    Each Interval should have the spatial extent of a single empty parking
    spot.
    """
    
    # Your code here

    first_frame = bounding_boxes[reference_video].filter(
        lambda interval: interval['t1'] == 0 and interval['payload']['class'] == 'car'
        ).get_intervals()

    video_lengths = {
        key: bounding_boxes[key].get_intervals()[-1]['t2']
        for key in bounding_boxes
    }

    first_duplicated = IntervalSetMapping({
        key: IntervalSet([
            Interval(Bounds3D(
                t1 = t,
                t2 = t + 30,
                x1 = interval['x1'],
                x2 = interval['x2'],
                y1 = interval['y1'],
                y2 = interval['y2']
            ))
            for t in range(0, int(video_lengths[key]), 30)
            for interval in first_frame
        ])
        for key in bounding_boxes
    })
    
    this_frame = bounding_boxes.filter(lambda interval: True)
    
    parking_spots = first_duplicated.minus(
        this_frame,
        predicate = and_pred(
            Bounds3D.T(equal()),
            Bounds3D.X(overlaps()),
            Bounds3D.Y(overlaps()),
            iou_at_least(0.1)
        ),
        window = 0.0,
        progress_bar = True
    )
    
    return  parking_spots

In [139]:
def evaluate_on_dev_modified():
    empty_parking_spaces = detect_empty_parking_spaces_modified(bounding_boxes, 0)
    
    # Visualize the predictions the first row will be your predictions, second
    # row will be the ground truth
    widget = visualize_boxes([empty_parking_spaces, parking_space_gt])
    display(widget)
    
    # Compute average precision on the dev set
    ap = compute_ap(empty_parking_spaces, parking_space_gt)
    
    print('Average precision: ', ap)

In [140]:
evaluate_on_dev_modified()




  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.84s/it]


100%|██████████| 2/2 [00:18<00:00,  7.14s/it]

VGridWidget(vgrid_spec={'compressed': True, 'data': b'x\x9c\xdd\x9cOo\x1b\xc9\x11\xc5\xbf\x8a\xa1s`\xcd\xf4\xf…




  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 38.16it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  3.78it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  3.76it/s]

Average precision:  0.9120812507408258


In [141]:
evaluate_on_test(detect_empty_parking_spaces_modified)




  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 649.22it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 67.12it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.95s/it]


100%|██████████| 2/2 [00:18<00:00,  7.19s/it]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00, 80.44it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  8.64it/s]


  0%|          | 0/2 [00:00<?, ?it/s]


100%|██████████| 2/2 [00:00<00:00,  8.97it/s]

Average precision:  0.9545454545454546
